In [39]:
import pickle

from detoxify import Detoxify
from tqdm import tqdm

from project.pipeline.data_loader import DataLoader

In [30]:
# create model
detoxify_model = Detoxify('original')
print("Model loaded!")

Model loaded!


In [31]:
print(next(detoxify_model.model.parameters()).device)

cpu


In [32]:
detoxify_model.model.to('cuda')
print(next(detoxify_model.model.parameters()).device)

cuda:0


In [33]:
number_param = sum(param.numel() for param in detoxify_model.model.parameters())
print(f"{number_param // (1024 ** 2)}M params")

104M params


In [34]:
# load the data
data_path = "../data/train_all_tasks.csv"
data_loader = DataLoader(data_path=data_path, batch_size=16)

In [37]:
original_results = []

for batch in tqdm(data_loader):
    # extract text
    text = list(batch['text'].values)

    # get results from toxic bert
    results = detoxify_model.predict(text)
    original_results.append(results)

100%|██████████| 875/875 [01:32<00:00,  9.44it/s]


In [38]:
# results are a list[dict[str, list[float]]]
# convert to dict[str, list[float]]

# get the keys
keys = list(original_results[0].keys())

# create a dict
original_results_dict = {key: [] for key in keys}

# iterate the results
for result in original_results:
    for key in keys:
        original_results_dict[key].extend(result[key])

In [40]:
with open('../cached/toxic_bert_results.pickle', 'wb') as handle:
    pickle.dump(original_results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)